# Using NLP on Educational Reform Policies to Predict Educational Outcome
### Hyperparameter Tuning and Cross Validation

#### Import Statements

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline

In [ ]:
# Load preprocessed df
final_df = pd.read_csv('../Data/final_pisa_werd_merged.csv')

### Word Embeddings

#### TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(final_df['reform_description_clean'])

print(tfidf_features.shape)

#### Word2Vec

In [ ]:
final_df['tokens'] = final_df['reform_description_clean'].apply(lambda x: x.split())

word2vec_model = gensim.models.Word2Vec(final_df['tokens'])

def document_vector(doc):
    doc = [word for word in doc if word in word2vec_model.wv.index_to_key]
    if not doc:
        return np.zeros(100)
    return np.mean(word2vec_model.wv[doc], axis=0)

word2vec_features = np.vstack(final_df['tokens'].apply(document_vector))

print("Word2Vec features shape:", word2vec_features.shape)

#### Doc2Vec

In [ ]:
final_df['tokens'] = final_df['reform_description_clean'].apply(lambda x: x.split())

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(final_df['tokens'])]

doc2vec_model = Doc2Vec(documents)

doc2vec_features = np.array([doc2vec_model.infer_vector(doc.words) for doc in documents])

print("Doc2Vec features shape:", doc2vec_features.shape)

#### BERT - BASE

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

bert_base_features = np.vstack(final_df['reform_description_clean'].apply(encode_text))

print("BERT features shape:", bert_base_features.shape)

#### BERT - Large

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained('bert-large-uncased')

def encode_text_large(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

bert_large_features = np.vstack(final_df['reform_description_clean'].apply(encode_text_large))

print("BERT features shape:", bert_large_features.shape)

## Hyperparameter Tuning

#### TF-IDF Hyperparameter Tuning

In [ ]:
X = tfidf_features
y = final_df['Mean_Last_PISA_Score'].values

X_train, X_test, y_train, y_test = train_test_split(final_df['reform_description_clean'], y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_df': [0.25, 0.5, 0.75, 1.0],
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

best_tfidf_params = grid_search.best_params_
print("Best TF-IDF parameters:", best_tfidf_params)

best_tfidf_vectorizer = TfidfVectorizer(**{k.replace("tfidf__", ""): v for k, v in best_tfidf_params.items()})
X_train_tfidf = best_tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = best_tfidf_vectorizer.transform(X_test)

# Fit the RandomForest with the best TF-IDF parameters
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_tfidf, y_train)
predictions = rf_model.predict(X_test_tfidf)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print("Test MAE:", mae)
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test R²:", r2)

#### Word2Vec Hyperparameter Tuning

In [ ]:
X = word2vec_features
y = final_df['Mean_Last_PISA_Score'].values

final_df['tokens'] = final_df['reform_description_clean'].apply(lambda x: x.split())

vector_sizes = [50, 75, 100, 125, 150, 175, 200, 225]
windows = [1, 2, 3, 4, 5, 6, 7, 8]
min_counts = [1, 2, 3]

word2vec_results = []

for vector_size in vector_sizes:
    for window in windows:
        for min_count in min_counts:
            model = gensim.models.Word2Vec(final_df['tokens'], vector_size=vector_size, window=window, min_count=min_count, workers=4)
            X = np.vstack(final_df['tokens'].apply(lambda doc: np.mean([model.wv[word] for word in doc if word in model.wv], axis=0)))
            y = final_df['Mean_Last_PISA_Score']
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_model.fit(X_train, y_train)
            predictions = rf_model.predict(X_test)
            mae = mean_absolute_error(y_test, predictions)
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, predictions)
            
            word2vec_results.append({
                'vector_size': vector_size,
                'window': window,
                'min_count': min_count,
                'MAE': mae,
                'MSE': mse,
                'RMSE': rmse,
                'R2': r2
            })


word2vec_results_df = pd.DataFrame(word2vec_results)

def display_best_config(results_df, metric, minimize=True):
    if minimize:
        best_row = results_df.loc[results_df[metric].idxmin()]
    else:
        best_row = results_df.loc[results_df[metric].idxmax()]
    print(f"Best configuration based on {metric} ({'Min' if minimize else 'Max'}imize):")
    print(best_row)

metrics = ['MAE', 'MSE', 'RMSE', 'R2']

print("\nBest Word2Vec Configurations:")
for metric in metrics:
    display_best_config(word2vec_results_df, metric, minimize=(metric != 'R2'))

#### Doc2Vec Hyperparameter Tuning

In [ ]:
X = doc2vec_features
y = final_df['Mean_Last_PISA_Score'].values

final_df['tokens'] = final_df['reform_description_clean'].apply(lambda x: x.split())

doc2vec_results = []
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(final_df['tokens'])]

for vector_size in vector_sizes:
    for window in windows:
        for min_count in min_counts:
            model = Doc2Vec(documents, vector_size=vector_size, window=window, min_count=min_count, workers=4)
            
            X = np.array([model.infer_vector(doc.words) for doc in documents])
            y = final_df['Mean_Last_PISA_Score']
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_model.fit(X_train, y_train)
            
            predictions = rf_model.predict(X_test)
            mae = mean_absolute_error(y_test, predictions)
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, predictions)
            
            doc2vec_results.append({
                'vector_size': vector_size,
                'window': window,
                'min_count': min_count,
                'MAE': mae,
                'MSE': mse,
                'RMSE': rmse,
                'R2': r2
            })

doc2vec_results_df = pd.DataFrame(doc2vec_results)

print("\nBest Doc2Vec Configurations:")
for metric in metrics:
    display_best_config(doc2vec_results_df, metric, minimize=(metric != 'R2'))


#### BERT-Base Hyperparameter Tuning

In [ ]:
X = bert_base_features
y = final_df['Mean_Last_PISA_Score'].values
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

def encode_text(texts, max_len, padding, truncation):
    max_len = min(max_len, tokenizer.model_max_length)
    inputs = tokenizer(texts, return_tensors='pt', padding=padding, truncation=truncation, max_length=max_len)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

parameters = {
    'max_length': [128, 256, 512],
    'padding': ['max_length'],
    'truncation': [True]
}

bert_results = []

for max_len, pad, trunc in product(parameters['max_length'], parameters['padding'], parameters['truncation']):
    X = np.vstack(final_df['reform_description_clean'].apply(lambda text: encode_text([text], max_len, pad, trunc)))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    predictions = rf_model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    bert_results.append({
        'max_length': max_len,
        'padding': pad,
        'truncation': trunc,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    })

bert_results_df = pd.DataFrame(bert_results)

print("\nBest BERT Configurations:")
metrics = ['MAE', 'MSE', 'RMSE', 'R2']
for metric in metrics:
    best_config = bert_results_df.loc[bert_results_df[metric].idxmin()] if metric != 'R2' else bert_results_df.loc[bert_results_df[metric].idxmax()]
    print(f"Best {metric}: {best_config.to_dict()}")

#### BERT-Large Hyperparameter Tuning

In [ ]:
X = bert_large_features
y = final_df['Mean_Last_PISA_Score'].values

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained('bert-large-uncased')
model.eval()

def encode_text(texts, max_len, padding, truncation):
    max_len = min(max_len, tokenizer.model_max_length)
    inputs = tokenizer(texts, return_tensors='pt', padding=padding, truncation=truncation, max_length=max_len)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

parameters = {
    'max_length': [128, 256, 512],
    'padding': ['max_length'],
    'truncation': [True]
}

bert_large_results = []

for max_len, pad, trunc in product(parameters['max_length'], parameters['padding'], parameters['truncation']):
    X = np.vstack(final_df['reform_description_clean'].apply(lambda text: encode_text([text], max_len, pad, trunc)))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    predictions = rf_model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    bert_large_results.append({
        'max_length': max_len,
        'padding': pad,
        'truncation': trunc,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    })

bert_large_results_df = pd.DataFrame(bert_large_results)

print("\nBest BERT Large Configurations:")
metrics = ['MAE', 'MSE', 'RMSE', 'R2']
for metric in metrics:
    best_config = bert_large_results_df.loc[bert_large_results_df[metric].idxmin()] if metric != 'R2' else bert_large_results_df.loc[bert_large_results_df[metric].idxmax()]
    print(f"Best {metric}: {best_config.to_dict()}")

### Hyperparameter Tuning Regression Models

#### Load the best word embedding configuration

In [ ]:
final_df['tokens'] = final_df['reform_description_clean'].apply(lambda x: x.split())

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(final_df['tokens'])]

doc2vec_model = Doc2Vec(documents, vector_size=175, window=6, min_count=3, workers=4)

doc2vec_best = np.array([doc2vec_model.infer_vector(doc.words) for doc in documents])

print("Doc2Vec features shape:", doc2vec_best.shape)

#### Hyperparameter Tuning Random Forest

In [ ]:
X = doc2vec_best
y = final_df['Mean_Last_PISA_Score'].values

param_grid = {
    'n_estimators': [50, 75, 100, 125, 150, 175, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

grid_search.fit(X, y)

print("Best parameters:", grid_search.best_params_)
print("Best MAE score:", -grid_search.best_score_)

#### Cross Validation Random Forest

In [ ]:
model = RandomForestRegressor(n_estimators=75, random_state=42, max_depth=10, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_jobs=-1)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)

mae_scores = cross_val_score(model, X, y, cv=kf, scoring=mae_scorer)
mse_scores = cross_val_score(model, X, y, cv=kf, scoring=mse_scorer)
r2_scores = cross_val_score(model, X, y, cv=kf, scoring=r2_scorer)

mae_scores = -mae_scores
mse_scores = -mse_scores
rmse_scores = np.sqrt(mse_scores)

print("MAE scores from each fold: ", mae_scores)
print("Average MAE: ", np.mean(mae_scores))
print("Standard Deviation of MAE: ", np.std(mae_scores))

print("MSE scores from each fold: ", mse_scores)
print("Average MSE: ", np.mean(mse_scores))
print("Standard Deviation of MSE: ", np.std(mse_scores))

print("RMSE scores from each fold: ", rmse_scores)
print("Average RMSE: ", np.mean(rmse_scores))
print("Standard Deviation of RMSE: ", np.std(rmse_scores))

print("R² scores from each fold: ", r2_scores)
print("Average R²: ", np.mean(r2_scores))
print("Standard Deviation of R²: ", np.std(r2_scores))

#### Hyperparameter Tuning XGBoost

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.9, 1.0]
}

xgb_model = xgb.XGBRegressor(random_state=42)

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
grid_search.fit(X, y)

best_params = grid_search.best_params_
print("Best parameters:", best_params)
print("Best MAE score:", -grid_search.best_score_)

#### Cross Validation XGBoost

In [ ]:
best_xgb_model = xgb.XGBRegressor(**best_params, random_state=42, n_jobs=-1)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)

mae_scores = cross_val_score(best_xgb_model, X, y, cv=kf, scoring=mae_scorer)
mse_scores = cross_val_score(best_xgb_model, X, y, cv=kf, scoring=mse_scorer)
r2_scores = cross_val_score(best_xgb_model, X, y, cv=kf, scoring=r2_scorer)

mae_scores = -mae_scores
mse_scores = -mse_scores
rmse_scores = np.sqrt(mse_scores)

print("MAE scores from each fold: ", mae_scores)
print("Average MAE: ", np.mean(mae_scores))
print("Standard Deviation of MAE: ", np.std(mae_scores))

print("MSE scores from each fold: ", mse_scores)
print("Average MSE: ", np.mean(mse_scores))
print("Standard Deviation of MSE: ", np.std(mse_scores))

print("RMSE scores from each fold: ", rmse_scores)
print("Average RMSE: ", np.mean(rmse_scores))
print("Standard Deviation of RMSE: ", np.std(rmse_scores))

print("R² scores from each fold: ", r2_scores)
print("Average R²: ", np.mean(r2_scores))
print("Standard Deviation of R²: ", np.std(r2_scores))

#### Hyperparameter Tuning SVR

In [ ]:
pipeline = make_pipeline(MaxAbsScaler(), SVR())

param_grid = {
    'svr__C': [0.1, 1, 10, 100],
    'svr__gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'svr__kernel': ['rbf', 'linear', 'poly'],
    'svr__epsilon': [0.1, 0.2, 0.5]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=2)
grid_search.fit(X, y)

best_params = grid_search.best_params_
print("Best parameters:", best_params)
print("Best MAE score:", -grid_search.best_score_)

#### Cross Validation SVR

In [ ]:
best_svm_params = {k.replace('svr__', ''): v for k, v in best_params.items()}

best_svm_model = make_pipeline(MaxAbsScaler(), SVR(**best_svm_params))

kf = KFold(n_splits=10, shuffle=True, random_state=42)

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)

mae_scores = cross_val_score(best_svm_model, X, y, cv=kf, scoring=mae_scorer)
mse_scores = cross_val_score(best_svm_model, X, y, cv=kf, scoring=mse_scorer)
r2_scores = cross_val_score(best_svm_model, X, y, cv=kf, scoring=r2_scorer)

mae_scores = -mae_scores
mse_scores = -mse_scores
rmse_scores = np.sqrt(mse_scores)

print("MAE scores from each fold: ", mae_scores)
print("Average MAE: ", np.mean(mae_scores))
print("Standard Deviation of MAE: ", np.std(mae_scores))

print("MSE scores from each fold: ", mse_scores)
print("Average MSE: ", np.mean(mse_scores))
print("Standard Deviation of MSE: ", np.std(mse_scores))

print("RMSE scores from each fold: ", rmse_scores)
print("Average RMSE: ", np.mean(rmse_scores))
print("Standard Deviation of RMSE: ", np.std(rmse_scores))

print("R² scores from each fold: ", r2_scores)
print("Average R²: ", np.mean(r2_scores))
print("Standard Deviation of R²: ", np.std(r2_scores))